# DataFrame

**Arquivo utilizado**:  *exemplo_alunos_final.xlsx*

**Fonte de consulta: https://pandas.pydata.org/pandas-docs/stable/index.html**


## Agrupando dados

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_excel('exemplo_alunos_final.xlsx')

In [ ]:
df.head()

In [ ]:
df.groupby('Curso').count()

In [ ]:
df.groupby(['Curso','Resultado']).count()

In [ ]:
colunas = ['Faltas', 'Media']
df.groupby('Curso')[colunas].mean()

In [ ]:
df.groupby(['Curso','Ingresso'])['Media'].mean()

In [ ]:
grouped = df.groupby(['Curso','Ingresso'])
grouped

In [ ]:
grouped['Media'].mean()

### Selecionando um grupo

In [ ]:
grouped2 = df.groupby(['Curso','Resultado'])
grouped2

In [ ]:
grouped2.get_group(('Atuária','Aprovado'))

# #Alternativamente (usando comandos de seleção)
# df.loc[(df.Curso=='Atuária') & (df.Resultado=='Aprovado')].describe()

In [ ]:
df.groupby(['Curso','Resultado']).get_group(('Secretariado','Recuperação'))

### Funções agregadoras

Depois que o objeto GroupBy tiver sido criado, podemos executar computação nos dados agrupados a partir de **funções de agregação**. As funções de agregação são aquelas que reduzem a dimensão dos objetos retornados. Algumas funções de agregação comuns estão tabuladas abaixo:

|Função| Descrição|
|---|---|
|mean() | Média dos grupos|
|sum() | Soma dos valores do grupo|
|size() | Tamanho dos grupos|
|count() | Total de observações do grupo|
|std() | Desvio padrão dos grupos|
|var() | Variância dos grupos|
|sem() | Erro padrão da média dos grupos|
|describe() | Estatísticas descritivas dos grupos|
|first() | Retorna os primeiros valores dos grupos|
|last() | Retorna os últimos dos valores dos grupos|
|nth() | Retorna o n-ésimo valor (ou n-ésimos, se for uma lista)|
|min() | Retorna o menor dos valores do grupo|
|max() | Calcular o máximo dos valores do grupo|


*As funções de agregação acima excluirão os valores de NA

In [ ]:
grouped2.aggregate('mean')

# #Alternativamente
# grouped2.mean()

In [ ]:
grouped2 = df.groupby(['Curso','Resultado'], as_index=False)
grouped2.aggregate('mean')

# #Alternativamente
# df.groupby(['Curso','Resultado']).mean().reset_index()

In [ ]:
grouped2.size()

In [ ]:
grouped2.count()

In [ ]:
grouped2 = df.groupby(['Curso','Resultado'])
grouped2['Media'].describe()

#### Aplicando múltiplas funções de uma só vez

In [ ]:
grouped2['Media'].aggregate(['count', 'mean'])

In [ ]:
stats = ['sum', 'mean', 'std']
nomes = {'sum': 'soma','mean': 'média','std': 'desvio padrão'}

grouped2['Media'].aggregate(stats).rename(columns=nomes)

In [ ]:
grouped2['Media', 'Faltas'].aggregate(stats).rename(columns=nomes)

Podemos aplicar diferentes funções de agregação para diferentes colunas.

In [ ]:
grouped2.aggregate({'Matricula':'count', 'Media':'mean'}).rename(columns={'Matricula':'Alunos'})

## Remodelando a Tabela de Dados 
### \[Cruzando variáveis em linhas e colunas\]

Os dados são frequentemente armazenados no chamado formato “empilhado”:

In [ ]:
tabela = df.groupby(['Curso','Resultado'], as_index=False).agg({'Faltas':'mean','Media':'mean'})
tabela = tabela[tabela.Resultado.isin(['Aprovado','Recuperação'])]
tabela

Suponha que desejamos reorganizar esta tabela, de modo a apresentar as *categorias de* **Resultados** nas colunas. Neste caso, usamos o método DataFrame.pivot():

In [ ]:
tabela.pivot(index='Curso',columns='Resultado')

In [ ]:
tabela.pivot(index='Curso',columns='Resultado',values='Media') 

# #Alternativamente
# tabela.pivot(index='Curso',columns='Resultado')['Media']

In [ ]:
tabela2 = df.groupby(['Curso','Ingresso','Resultado'], as_index=False).agg({'Faltas':'mean','Media':'mean'})
tabela2 = tabela2[tabela2.Resultado.isin(['Aprovado','Recuperação'])]
tabela2

In [ ]:
try: tabela2.pivot(index='Curso',columns='Resultado')['Media']
except ValueError: print('ValueError: Index contains duplicate entries, cannot reshape.')    

In [ ]:
tabela2.pivot_table(index=['Curso'],columns='Resultado',aggfunc=np.mean)['Media']

# #Alternativamente
# tabela2_ = tabela2.groupby(['Curso','Resultado'], as_index=False).agg({'Faltas':'mean','Media':'mean'})
# tabela2_.pivot(index='Curso',columns='Resultado')['Media']

In [ ]:
tabela2.pivot_table(index=['Curso','Ingresso'],columns='Resultado')['Media']

In [ ]:
tabela2.pivot_table(index='Curso',columns=['Ingresso','Resultado'], values='Media')

In [ ]:
tabela3 = tabela2.pivot_table(index=['Curso','Ingresso'],columns='Resultado')
tabela3

**Selecionando dados**

In [ ]:
tabela3.Media

In [ ]:
tabela3.Media.Recuperação

In [ ]:
tabela3.Media.Recuperação['Administração']

In [ ]:
tabela3.Media.Recuperação['Administração'][2017]

**Criando novas colunas**

In [ ]:
tabela4 = tabela3.copy()
tabela4['Diferença'] = tabela4.Media.Aprovado - tabela4.Media.Recuperação
tabela4

In [ ]:
tabela4.loc['Administração', 2017].Media.Aprovado

# #Alternativamente:
# tabela5.loc['Administração', 2017]['Media']['Aprovado']
# #ou ainda:
# tabela5.loc['Administração', 2017].Media['Aprovado']

**Empilhando os dados:**

In [ ]:
tabela5 = tabela4.stack()
tabela5

In [ ]:
tabela5.loc['Administração', 2017, 'Aprovado']['Media']

In [ ]:
tabela5.reset_index(inplace=True)
tabela5

In [ ]:
tabela5.loc[(tabela5.Curso=='Administração') & (tabela5.Ingresso==2017) & (tabela5.Resultado=='Aprovado'), 'Media']

In [ ]:
df.head()

### Exemplo:
### Programa que retorna uma tabela de pivot contendo uma estatística para uma variável numérica

In [ ]:
print('IMPRESSÃO DE TABELA PIVOT.\n')
print('Variáveis disponíveis: ')
for i in df.columns:
    print(i, end=", ")

variavel_linha = input('Variável categórica (linhas): ')
variavel_coluna = input('Variável categórica (colunas): ')
variavel_numerica = input('Variável numérica: ')
estat = int(input('Estatistica desejada (1 - média, 2 - Máximo, 3 - Mínimo): '))

print('\n')
if estat==1: print(df.pivot_table(index=[variavel_linha],columns=[variavel_coluna],aggfunc=np.mean)[variavel_numerica])
elif estat==2: print(df.pivot_table(index=[variavel_linha],columns=[variavel_coluna],aggfunc=np.max)[variavel_numerica])
elif estat==3: print(df.pivot_table(index=[variavel_linha],columns=[variavel_coluna],aggfunc=np.min)[variavel_numerica])
else: print('Estatistica inválida.')